# (공부) xgboost 재현

신록예찬  
2024-08-23

# Imports

In [2]:
import xgboost as xgb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.tree
#---#
import warnings
warnings.filterwarnings('ignore')
#---#
import matplotlib.animation
import IPython

# Data

In [3]:
np.random.seed(1)
temp = pd.read_csv('https://raw.githubusercontent.com/guebin/DV2022/master/posts/temp.csv').iloc[:,3].to_numpy()[:80]
temp.sort()
eps = np.random.randn(80)*3 # 오차
icecream_sales = 20 + temp * 2.5 #+ eps 
df_train = pd.DataFrame({'temp':temp,'sales':icecream_sales})
df_train

80 rows × 2 columns

# 적합

## 데이터

In [11]:
# step1 
X = df_train[['temp']]
X = np.array(X).astype(np.float32)
y = df_train['sales']
y = np.array(y).astype(np.float32)


## 코드1

In [19]:
## 코드1 
predictr = xgb.XGBRegressor(
    n_estimators=N,         # 부스팅 라운드 수
    learning_rate=0.3,         # 학습률
    max_depth=6,               # 최대 깊이
    min_child_weight=1,        # 최소 자식 가중치
    subsample=1.0,              # 데이터 샘플링 비율
    tree_method='exact',
    reg_lambda= 0.0
)
predictr.fit(X,y)
yhat = predictr.predict(X) 

## 코드2

In [25]:
## 코드2 
yhat2 = np.full(y.shape, np.mean(y), dtype=np.float32)
num_boosting_rounds = 100
for i in range(num_boosting_rounds):
    residuals = y - yhat2
    ## 코드1 
    predictr = xgb.XGBRegressor(
        n_estimators=1,         # 부스팅 라운드 수
        learning_rate=0.3,         # 학습률
        max_depth=6,               # 최대 깊이
        min_child_weight=1,        # 최소 자식 가중치
        subsample=1.0,              # 데이터 샘플링 비율
        tree_method='exact',
        reg_lambda= 0.0
    )
    predictr.fit(X,residuals)
    new_pred = predictr.predict(X) 
    yhat2 += new_pred

## 비교

In [26]:
## 코드1,2 비교
diff = ((yhat - yhat2) ** 2).sum()
print(diff)

6.274422e-08